In [ ]:
%mkdir -p /content/drive/MyDrive/GSE124703
%cd /content/drive/MyDrive/GSE124703

/content/drive/MyDrive/GSE124703


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE124703&format=file" -O GSE124703_RAW.tar
!tar -xvf GSE124703_RAW.tar

--2024-08-03 14:20:49--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE124703&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1091307520 (1.0G) [application/x-tar]
Saving to: ‘GSE124703_RAW.tar’

GSE124703_RAW.tar   100%[===================>]   1.02G  31.9MB/s    in 39s     

2024-08-03 14:21:28 (27.0 MB/s) - ‘GSE124703_RAW.tar’ saved [1091307520/1091307520]

GSM3543612_D14_C1_read_counts.txt.gz
GSM3543613_D14_C2_read_counts.txt.gz
GSM3543614_D21_C1_read_counts.txt.gz
GSM3543615_D21_C2_read_counts.txt.gz
GSM3543616_D28_C1_read_counts.txt.gz
GSM3543617_D28_C2_read_counts.txt.gz
GSM3543618_iPSC_1_barcodes.tsv.gz
GSM3543618_iPSC_1_genes.tsv.gz
GSM3543618_iPSC_1_matrix.mtx.gz
GSM3543619_iPSC_2_barcodes.tsv.gz
GSM3543619_iPSC_2_genes.tsv.gz
GSM3543619_iPSC_2_matrix.mtx.gz
GSM3543620_i

In [ ]:
!zcat GSM3543625_neuron_2_sgRNA_mapping.txt.gz | head

cell	barcode	read_count	umi_count
TCTCTAAAGACAGACC-1	GCACCTCCAGATCTCACTAG	0	0
TCTCTAAAGACAGACC-1	GTTGGCCATAAAGCCTGAGG	0	0
TCTCTAAAGACAGACC-1	GTGGCCTTACGAAGGAGCAG	2	1
TCTCTAAAGACAGACC-1	GGCGACGACACCACGATTCA	0	0
TCTCTAAAGACAGACC-1	unprocessed_TGTTGGTGGGGGCGCCACCCGCGGGGTTTA	0	0
TCTCTAAAGACAGACC-1	unprocessed_TGTTGGCGTGTAACGGCGCTGTGAGGTTTA	0	0
TCTCTAAAGACAGACC-1	GCGGCTCGGCTGTAAGGAGG	0	0
TCTCTAAAGACAGACC-1	GTCCCTTTCCTGTCAGACTG	0	0
TCTCTAAAGACAGACC-1	GGCGGCCGCAGCACAAAAGG	0	0


In [ ]:
!zcat GSM3543612_D14_C1_read_counts.txt.gz | head

ENSG00000000003	45
ENSG00000000005	0
ENSG00000000419	62
ENSG00000000457	18
ENSG00000000460	4
ENSG00000000938	1
ENSG00000000971	5
ENSG00000001036	27
ENSG00000001084	28
ENSG00000001167	14


In [ ]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
def read_10x_mtx_legacy(prefix):
    adata = sc.read_mtx(f"{prefix}matrix.mtx.gz").T
    barcodes = pd.read_csv(f"{prefix}barcodes.tsv.gz", header=None, sep='\t')
    barcodes.columns = ['cell_barcode']
    adata.obs.index = barcodes['cell_barcode']
    genes = pd.read_csv(f"{prefix}genes.tsv.gz", header=None, sep='\t', index_col=1)
    genes.index.name = 'gene_name'
    genes.columns = ['gene_id']
    adata.var = genes
    return adata

In [ ]:
import numpy as np

def assign_sgRNAs(df):
    df = df.reset_index().rename(columns={'index': 'cell'})

    df = df[df['umi_count'] > 0]

    grouped = df.groupby(['cell', 'barcode'])['umi_count'].sum().reset_index()

    cell_stats = grouped.groupby('cell')['umi_count'].agg(['mean', 'std'])
    cell_stats['threshold'] = cell_stats['mean'] + (4 * cell_stats['std'])

    grouped = grouped.merge(cell_stats, left_on='cell', right_index=True, suffixes=('', '_cell'))

    grouped['is_assigned'] = grouped['umi_count'] > grouped['threshold']

    def combine_sgRNAs(group):
        assigned = group[group['is_assigned']]['barcode']
        return '+'.join(assigned) if len(assigned) > 0 else np.nan

    assigned = grouped.groupby('cell').apply(combine_sgRNAs)
    assigned = assigned.rename('assigned_sgRNA')

    df = df.merge(assigned, on='cell', how='left')

    df_combined = df.groupby('cell').agg({
        'barcode': lambda x: '+'.join(sorted(set(x))),
        'read_count': 'sum',
        'umi_count': 'sum',
        'assigned_sgRNA': 'first'
    }).reset_index()

    df_combined = df_combined.set_index('cell')

    return df_combined


def assign_genes(df, sg_ref):
    gene_dict = sg_ref['gene_name'].to_dict()

    def assign(sgRNAs):
        if pd.isna(sgRNAs):
            return 'non-targeting'
        genes = [gene_dict.get(sgRNA.strip(), 'unknown') for sgRNA in sgRNAs.split('+')]
        return '+'.join(sorted(set(genes)))

    df['perturbation_name'] = df['assigned_sgRNA'].apply(assign)
    return df

In [ ]:
sg_ref = pd.read_excel('./1-s2.0-S0896627319306403-mmc3.xlsx', index_col=2)
sg_ref['gene_name'] = sg_ref['sgRNA_short name'].str.split('_').str[0]
sg_ref.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,sgRNA_short name,sgRNA_long name,Day 10 iPSC_rep1,Day 10 iPSC_rep2,Day 14 neuron_rep1,Day 14 neuron_rep2,Day 28 neuron_rep1,Day 14 co-culture neuron_rep1,Day 14 co-culture neuron_rep2,gene_name
protospacer sequence,,,,,,,,,,
GCGGCGACCCTAGGAGAGGT,AARS_sgRNA1,AARS_+_70323318.23-P1P2_h1_Top5,-2.652234,-4.969276,-3.186770,-0.442845,-2.695419,-1.174968,-0.971148,AARS
GGGGACGGCGACCCTAGGAG,AARS_sgRNA2,AARS_+_70323322.23-P1P2_h1_Top5,-1.765477,-0.853395,-1.627589,-1.167728,-1.219046,-1.191816,-0.589689,AARS
GAAAAGTTGGGCCGAAGGAG,ACTR2_sgRNA1,ACTR2_-_65454953.23-P1P2_h1_Top5,-3.391390,-3.828490,-4.638436,-4.759581,-4.908412,-4.664601,-4.256598,ACTR2
GGGGGCCGGGAAGACGCAAG,ACTR2_sgRNA2,ACTR2_-_65454971.23-P1P2_h1_Top5,-3.910616,-3.968495,-5.613332,-5.418322,-5.726655,-5.810321,-5.354803,ACTR2
GTAAGGGAGGCAAGGACGGA,ARPC2_sgRNA1,ARPC2_+_219082121.23-P1_h1_Top5,-3.408098,-3.535386,-4.158646,-4.306514,-4.245050,-4.190037,-3.884749,ARPC2


In [ ]:
import pandas as pd
import numpy as np

keys = ['GSM3543618_iPSC_1_', 'GSM3543619_iPSC_2_', 'GSM3543622_neuron_1_', 'GSM3543625_neuron_2']
sample_map = {
    'GSM3543618_iPSC_1_': 'GSM3543620_iPSC_1_',
    'GSM3543619_iPSC_2_': 'GSM3543621_iPSC_2_',
    'GSM3543622_neuron_1_': 'GSM3543624_neuron_1_',
    'GSM3543623_neuron_2_': 'GSM3543625_neuron_2_'
}
adatas = {}
for prefix, map in sample_map.items():
    print(f"Processing {prefix}")
    adata = read_10x_mtx_legacy(prefix)
    df = pd.read_csv(f'{map}sgRNA_mapping.txt.gz', sep='\t', index_col=0)
    df = assign_sgRNAs(df)
    df = assign_genes(df, sg_ref)
    df = df.loc[~df['perturbation_name'].str.contains('unknown')]

    adata = adata[df.index]
    adata.obs = df
    adata.obs_names_make_unique()
    adata.var_names_make_unique()
    adatas[prefix.rstrip('_')] = adata

adata = ad.concat(adatas, label='batch')
adata.obs

Processing GSM3543618_iPSC_1_


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Processing GSM3543619_iPSC_2_


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Processing GSM3543622_neuron_1_


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Processing GSM3543623_neuron_2_


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,barcode,read_count,umi_count,assigned_sgRNA,perturbation_name,batch
cell,,,,,,
AAACCTGAGAAACCAT-1,GACCCGAGACTGCTTCCCGG+GCACCTCCAGATCTCACTAG+GCCT...,22,13,None,non-targeting,GSM3543618_iPSC_1
AAACCTGAGAAACGAG-1,GACCCGAGACTGCTTCCCGG+GCCTGGCTCATGGCTGTGTG+GTGG...,8,3,None,non-targeting,GSM3543618_iPSC_1
AAACCTGAGACAAGCC-1,GACCGGATCGGTGACTGTGG+GCCTGGCTCATGGCTGTGTG+GCGG...,55,27,None,non-targeting,GSM3543618_iPSC_1
AAACCTGAGACAGACC-1,GCTCACCAAGCGTGACCGAG,1,1,None,non-targeting,GSM3543618_iPSC_1
AAACCTGAGACGACGT-1,GGCGCTGGCCGCAGTCTGAC,1,1,None,non-targeting,GSM3543618_iPSC_1
...,...,...,...,...,...,...
TTTGTCATCTGGCGAC-1,GACCCGAGACTGCTTCCCGG,1,1,None,non-targeting,GSM3543623_neuron_2
TTTGTCATCTGTCCGT-1,GTGCCGCGGTGACCTCACGG,1,1,None,non-targeting,GSM3543623_neuron_2
TTTGTCATCTTAACCT-1,GACCCGAGACTGCTTCCCGG+GACCGGATCGGTGACTGTGG+GCAC...,6,6,None,non-targeting,GSM3543623_neuron_2


In [ ]:
adata.obs['condition'] = adata.obs['perturbation_name'].apply(
    lambda x: 'non-targeting' if x == 'non-targeting' else 'targeting'
)

In [ ]:
adata = adata[~adata.obs['perturbation_name'].str.contains('unknown')]
adata

View of AnnData object with n_obs × n_vars = 478679 × 33752
    obs: 'barcode', 'read_count', 'umi_count', 'assigned_sgRNA', 'perturbation_name', 'batch', 'condition'

In [ ]:
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = adata.obs['batch'].apply(
    lambda x: 'induced pluripotent stem cell' if 'iPSC' in x else 'neuron'
)
adata.obs['organism'] = ORGANISMS[0]

<ipython-input-10-afdde53ffecc>:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['crispr_type'] = 'CRISPRi'
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [ ]:
adata.write_h5ad('./GSE124703_merged.h5ad')

In [ ]:
adata = sc.read_h5ad('./GSE124703_merged.h5ad')
adata

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 478679 × 33752
    obs: 'barcode', 'read_count', 'umi_count', 'assigned_sgRNA', 'perturbation_name', 'batch', 'condition', 'crispr_type', 'cancer_type', 'cell_type', 'organism'

In [ ]:
import pandas as pd
import numpy as np

keys = ['GSM3543618_iPSC_1_', 'GSM3543619_iPSC_2_', 'GSM3543622_neuron_1_', 'GSM3543625_neuron_2']
sample_map = {
    'GSM3543618_iPSC_1_': 'GSM3543620_iPSC_1_',
    'GSM3543619_iPSC_2_': 'GSM3543621_iPSC_2_',
    'GSM3543622_neuron_1_': 'GSM3543624_neuron_1_',
    'GSM3543623_neuron_2_': 'GSM3543625_neuron_2_'
}
adatas = {}
for prefix, map in sample_map.items():
    print(f"Processing {prefix}")
    barcodes = pd.read_csv(f"{prefix}barcodes.tsv.gz", header=None, sep='\t')
    df = pd.read_csv(f'{map}sgRNA_mapping.txt.gz', sep='\t', index_col=0)
    print(barcodes.shape, df.index.unique().shape)

Processing GSM3543618_iPSC_1_
(737280, 1) (146335,)
Processing GSM3543619_iPSC_2_
(737280, 1) (150824,)
Processing GSM3543622_neuron_1_
(737280, 1) (100993,)
Processing GSM3543623_neuron_2_
(737280, 1) (89734,)


In [16]:
adata.obs.loc[adata.obs['perturbation_name'].str.split('+').str.len() == 1, 'perturbation_name'].unique().shape

(27,)

In [19]:
adata.obs.loc[adata.obs['perturbation_name'].str.split('+').str.len() == 1, 'perturbation_name'].shape[0] / 27

17725.296296296296

In [18]:
adata.n_obs

478679